### ResNet
- VGG이후 더 깊은 Network에 대한 연구가 증가했지만, Network 깊이가 깊어질수록 오히려 accuracy가 떨어지는 문제가 있었다.
- 층이 깊어질 수록 계속해서 기울기가 0에 가까워지는 Gradient vanishing이 발생하기 때문이다.
<img src="./images/ResNet01.png" width="400" style="margin:30px; margin-left: 0">
- 이를 해결하고자 층을 만들되, Input 데이터와 결과가 동일하게 나올 수 있도록 하는 층을 연구하기 시작했다. 함수로 나타내면 H(x) = x이다. 
- 하지만 활성화 함수를 통과한 값을 기존 Input 데이터와 동일하게 만드는 것은 굉장히 복잡했기 때문에  
H(x) = F(x) + x 즉, F(x)를 0으로 만드는 F(x)에 포커스를 하게 된다.
- 이는 다음 레이어에 바로 적합되는 것이 아니라 원래 Output에서 자기자신을 빼는(잔차) F(x)의 mapping에 적합하도록 구성하는 것을 의미한다.
- 즉, residual mapping(잔차 매핑)이 기존의 mapping보다 optimize(최적화) 하기 쉽다는 것을 가정하게 된다.
- input은 x이고, Model인 F(x)라는 일련의 과정을 거치면서 자신(identity)인 x가 더해져서 output으로 F(x) + x 가 나오는 구조가 된다.
<img src="./images/ResNet02.png" style="margin:30px; margin-left: 0;">

- F(x)가 0이 되도록 학습하는 것은 단순히 0이 되는 것이 아니라 0으로 학습하면서 기존 CNN과 유사하게 입력값의 비선형적(Relu)인 feature를 학습한다. 
- Residual Block의 출력과 shortcut connection으로 전달받은 이전 출력 값을 Add하는 Identity Block의 출력을 Relu에 적용한다.
- 이를 통해, 깊은 네트워크에서 최소 Loss 최적화를 더 빠르고 효율적으로 수행할 수 있다.
- F(x) 형태의 모듈을 Residual(잔차) Block이라고 하고, shortcut connection으로 전달받은 자신인(identity)와 Add까지를 Identity Block라고 한다.

<img src="./images/ResNet03.png" width="400" style="margin:30px; margin-left: 0;">

In [4]:
from tensorflow.keras.layers import ZeroPadding2D, MaxPooling2D

def do_first_conv(input_tensor):
    # 7x7 Conv 연산 수행하여 feature map 생성하되 input_tensor 크기(image 크기)의 절반으로 생성.  filter 개수는 64개 
    # 224x224 를 input을 7x7 conv, strides=2로 112x112 출력하기 위해 Zero padding 적용. 
    x = ZeroPadding2D(padding=(3, 3), name='conv1_pad')(input_tensor)
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='valid', kernel_initializer='he_normal', name='conv')(x)
    x = BatchNormalization(axis=3, name='bn_conv1')(x)
    x = Activation('relu')(x)
    # 다시 feature map 크기를 MaxPooling으로 절반으로 만듬. 56x56으로 출력하기 위해 zero padding 적용. 
    x = ZeroPadding2D(padding=(1, 1), name='pool1_pad')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    
    return x

In [5]:
from tensorflow.keras.layers import Conv2D, Dense, BatchNormalization, Activation
from tensorflow.keras.layers import add, Add

# 여러 개의 block을 stage로 구분
# block은 동일 stage내에서 identity block을 구분
def identity_block(input_tensor, kernel_size, filters, stage, block):
    # filter1은 첫번째 1x1 filter 개수, filter2는 3x3 filter개수, filter3는 마지막 1x1 filter개수
    filter1, filter2, filter3 = filters
    # conv layer와 Batch normalization layer각각에 고유한 이름을 부여하기 위해 설정. 입력받은 stage와 block에 기반하여 이름 부여
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # 이전 layer에 입력 받은 input_tensor(함수인자로 입력받음)를 기반으로 첫번째 1x1 Conv->Batch Norm->Relu 수행. 
    # 첫번째 1x1 Conv에서 Channel Dimension Reduction 수행. filter1은 입력 input_tensor(입력 Feature Map) Channel 차원 개수의 1/4임. 
    x = Conv2D(filters=filter1, kernel_size=(1, 1), kernel_initializer='he_normal', name=conv_name_base+'2a')(input_tensor)
    # Batch Norm적용. 입력 데이터는 batch 사이즈까지 포함하여 4차원임(batch_size, height, width, channel depth)임
    # Batch Norm의 axis는 channel depth에 해당하는 axis index인 3을 입력.(무조건 channel이 마지막 차원의 값으로 입력된다고 가정. )
    x = BatchNormalization(axis=3, name=bn_name_base+'2a')(x)
    # ReLU Activation 적용. 
    x = Activation('relu')(x)
    
    # 두번째 3x3 Conv->Batch Norm->ReLU 수행
    # 3x3이 아닌 다른 kernel size도 구성 가능할 수 있도록 identity_block() 인자로 입력받은 kernel_size 이용. 
    # Conv 수행 출력 사이즈가 변하지 않도록 padding='same'으로 설정. filter 개수는 이전의 1x1 filter개수와 동일.  
    x = Conv2D(filters=filter2, kernel_size=kernel_size, padding='same', kernel_initializer='he_normal', name=conv_name_base+'2b')(x)
    x = BatchNormalization(axis=3, name=bn_name_base+'2b')(x)
    x = Activation('relu')(x)
    
    # 마지막 1x1 Conv->Batch Norm 수행. ReLU를 수행하지 않음에 유의.
    # filter 크기는 input_tensor channel 차원 개수로 원복
    x = Conv2D(filters=filter3, kernel_size=(1, 1), kernel_initializer='he_normal', name=conv_name_base+'2c')(x)
    x = BatchNormalization(axis=3, name=bn_name_base+'2c')(x)
    # Residual Block 수행 결과와 input_tensor를 합한다. 
    x = Add()([input_tensor, x])
    # 또는 x = add([x, input_tensor]) 와 같이 구현할 수도 있음. 

    # 마지막으로 identity block 내에서 최종 ReLU를 적용
    x = Activation('relu')(x)
    
    return x

In [6]:
def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    '''
    함수 입력 인자 설명
    input_tensor: 입력 tensor
    middle_kernel_size: 중간에 위치하는 kernel 크기. identity block내에 있는 두개의 conv layer중 1x1 kernel이 아니고, 3x3 kernel임. 
                        3x3 커널 이외에도 5x5 kernel도 지정할 수 있게 구성. 
    filters: 3개 conv layer들의 filter개수를 list 형태로 입력 받음. 첫번째 원소는 첫번째 1x1 filter 개수, 두번째는 3x3 filter 개수, 
             세번째는 마지막 1x1 filter 개수
    stage: identity block들이 여러개가 결합되므로 이를 구분하기 위해서 설정. 동일한 filter수를 가지는 identity block들을  동일한 stage로 설정.  
    block: 동일 stage내에서 identity block을 구별하기 위한 구분자
    strides: 입력 feature map의 크기를 절반으로 줄이기 위해서 사용. Default는 2이지만, 
             첫번째 Stage의 첫번째 block에서는 이미 입력 feature map이 max pool로 절반이 줄어있는 상태이므로 다시 줄이지 않기 위해 1을 호출해야함 
    ''' 
    
    # filters로 list 형태로 입력된 filter 개수를 각각 filter1, filter2, filter3로 할당. 
    # filter은 첫번째 1x1 filter 개수, filter2는 3x3 filter개수, filter3는 마지막 1x1 filter개수
    filter1, filter2, filter3 = filters
    # conv layer와 Batch normalization layer각각에 고유한 이름을 부여하기 위해 설정. 입력받은 stage와 block에 기반하여 이름 부여
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # 이전 layer에 입력 받은 input_tensor(함수인자로 입력받음)를 기반으로 첫번째 1x1 Conv->Batch Norm->Relu 수행. 
    # 입력 feature map 사이즈를 1/2로 줄이기 위해 strides인자를 입력  
    x = Conv2D(filters=filter1, kernel_size=(1, 1), strides=strides, kernel_initializer='he_normal', name=conv_name_base+'2a')(input_tensor)
    # Batch Norm적용. 입력 데이터는 batch 사이즈까지 포함하여 4차원임(batch_size, height, width, channel depth)임
    # Batch Norm의 axis는 channel depth에 해당하는 axis index인 3을 입력.(무조건 channel이 마지막 차원의 값으로 입력된다고 가정. )
    x = BatchNormalization(axis=3, name=bn_name_base+'2a')(x)
    # ReLU Activation 적용. 
    x = Activation('relu')(x)
    
    # 두번째 3x3 Conv->Batch Norm->ReLU 수행
    # 3x3이 아닌 다른 kernel size도 구성 가능할 수 있도록 identity_block() 인자로 입력받은 middle_kernel_size를 이용. 
    # Conv 수행 출력 사이즈가 변하지 않도록 padding='same'으로 설정. filter 개수는 이전의 1x1 filter개수와 동일.  
    x = Conv2D(filters=filter2, kernel_size=kernel_size, padding='same', kernel_initializer='he_normal', name=conv_name_base+'2b')(x)
    x = BatchNormalization(axis=3, name=bn_name_base+'2b')(x)
    x = Activation('relu')(x)
    
    # 마지막 1x1 Conv->Batch Norm 수행. ReLU를 수행하지 않음에 유의.
    # filter 크기는 input_tensor channel 차원 개수로 원복
    x = Conv2D(filters=filter3, kernel_size=(1, 1), kernel_initializer='he_normal', name=conv_name_base+'2c')(x)
    x = BatchNormalization(axis=3, name=bn_name_base+'2c')(x)
    
    # shortcut을 1x1 conv 수행, filter3가 입력 feature map의 filter 개수
    shortcut = Conv2D(filter3, (1, 1), strides=strides, kernel_initializer='he_normal', name=conv_name_base+'1')(input_tensor)
    shortcut = BatchNormalization(axis=3, name=bn_name_base+'1')(shortcut)
    
    # Residual Block 수행 결과와 1x1 conv가 적용된 shortcut을 합한다. 
    x = add([x, shortcut])
    
    # 마지막으로 identity block 내에서 최종 ReLU를 적용
    x = Activation('relu')(x)
    
    return x
    

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam , RMSprop 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler

def resnet(in_shape=(224, 224, 3), n_classes=10):
    input_tensor = Input(shape=in_shape)
    
    #첫번째 7x7 Conv와 Max Polling 적용.  
    x = do_first_conv(input_tensor)
    
#     conv_block에서 크기를 줄이고, identity_block에서는 크기를 그대로 가져간다.
    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')
    
    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

#     conv_block에서 크기를 줄이고, identity_block에서는 크기를 그대로 가져간다.
    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

#     conv_block에서 크기를 줄이고, identity_block에서는 크기를 그대로 가져간다.
    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
    
    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dropout(rate=0.5)(x)
    x = Dense(200, activation='relu', name='fc_01')(x)
    x = Dropout(rate=0.5)(x)
    output = Dense(n_classes, activation='softmax', name='fc_final')(x)
    
    model = Model(inputs=input_tensor, outputs=output, name='resnet50')
    model.summary()
    
    return model

In [8]:
model =  resnet(in_shape=(224,224,3), n_classes=10)

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv (Conv2D)               (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 bn_conv1 (BatchNormalizati  (None, 112, 112, 64)         256       ['conv[0][0]']                
 on)                                                                                       

                                                                                                  
 activation_8 (Activation)   (None, 28, 28, 64)           0         ['bn2c_branch2b[0][0]']       
                                                                                                  
 res2c_branch2c (Conv2D)     (None, 28, 28, 256)          16640     ['activation_8[0][0]']        
                                                                                                  
 bn2c_branch2c (BatchNormal  (None, 28, 28, 256)          1024      ['res2c_branch2c[0][0]']      
 ization)                                                                                         
                                                                                                  
 add_2 (Add)                 (None, 28, 28, 256)          0         ['activation_6[0][0]',        
                                                                     'bn2c_branch2c[0][0]']       
          

 ization)                                                                                         
                                                                                                  
 add_5 (Add)                 (None, 14, 14, 512)          0         ['activation_15[0][0]',       
                                                                     'bn3c_branch2c[0][0]']       
                                                                                                  
 activation_18 (Activation)  (None, 14, 14, 512)          0         ['add_5[0][0]']               
                                                                                                  
 res3d_branch2a (Conv2D)     (None, 14, 14, 128)          65664     ['activation_18[0][0]']       
                                                                                                  
 bn3d_branch2a (BatchNormal  (None, 14, 14, 128)          512       ['res3d_branch2a[0][0]']      
 ization) 

                                                                                                  
 res4c_branch2a (Conv2D)     (None, 7, 7, 256)            262400    ['activation_27[0][0]']       
                                                                                                  
 bn4c_branch2a (BatchNormal  (None, 7, 7, 256)            1024      ['res4c_branch2a[0][0]']      
 ization)                                                                                         
                                                                                                  
 activation_28 (Activation)  (None, 7, 7, 256)            0         ['bn4c_branch2a[0][0]']       
                                                                                                  
 res4c_branch2b (Conv2D)     (None, 7, 7, 256)            590080    ['activation_28[0][0]']       
                                                                                                  
 bn4c_bran

 ization)                                                                                         
                                                                                                  
 activation_38 (Activation)  (None, 7, 7, 256)            0         ['bn4f_branch2b[0][0]']       
                                                                                                  
 res4f_branch2c (Conv2D)     (None, 7, 7, 1024)           263168    ['activation_38[0][0]']       
                                                                                                  
 bn4f_branch2c (BatchNormal  (None, 7, 7, 1024)           4096      ['res4f_branch2c[0][0]']      
 ization)                                                                                         
                                                                                                  
 add_12 (Add)                (None, 7, 7, 1024)           0         ['activation_36[0][0]',       
          

 bn5c_branch2c (BatchNormal  (None, 4, 4, 2048)           8192      ['res5c_branch2c[0][0]']      
 ization)                                                                                         
                                                                                                  
 add_15 (Add)                (None, 4, 4, 2048)           0         ['activation_45[0][0]',       
                                                                     'bn5c_branch2c[0][0]']       
                                                                                                  
 activation_48 (Activation)  (None, 4, 4, 2048)           0         ['add_15[0][0]']              
                                                                                                  
 avg_pool (GlobalAveragePoo  (None, 2048)                 0         ['activation_48[0][0]']       
 ling2D)                                                                                          
          